# Deep Inception V3 for transfer learning

In [1]:
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers, models

In [3]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

2023-08-13 13:39:31.667726: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


87910968/87910968 [==============================] - 94s 1us/step


In [5]:
x = base_model.output
# let's add a fully-connected layer as first layer
x = layers.Dense(1024, activation='relu')(x)
# and a logistic layer with 200 classes as last layer
predictions = layers.Dense(200, activation='softmax')(x)
# model to train
model = models.Model(inputs=base_model.input, outputs=predictions)

In [6]:
# freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [7]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [8]:
# train the model on the new data for a few epochs
#model.fit_generator()

Then, we freeze the top inception layers and fine-tune the other inception layers. In this example, we
decide to freeze the first `172` layers (this is a tunable hyperparameter):

In [9]:
for layer in model.layers[:172]:
    layer.trainable = False

for layer in model.layers[172:]:
    layer.trainable = True

In [11]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from tensorflow.keras.optimizers import SGD

model.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.9), loss='categorical_crossentropy')

In [12]:
# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
#model.fit_generator()